In [6]:
from pymongo import MongoClient
import requests
from dotenv import load_dotenv
import os
import pandas as pd
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
from functools import reduce
import operator
import json
import re
from geopy.geocoders import Nominatim

In [7]:
df = pd.read_csv('df_final_clean3.csv', index_col=0)
df.dropna(how = "all", axis = 0, inplace = True)

In [8]:
df.head()

,name,address,type,ranking,price,direccion,comunidad,street,street_2,coordinates,street_3
indice,,,,,,,,,,,
413.0,El anciano rey de los vinos,"Calle Bailén 19, 28013 Madrid España",Mediterránea,#549 de 10.409 restaurantes en Madrid,€€ - €€€,Calle Bailén 19,Madrid España,Calle Bailén 19,"Calle Bailén 19, Madrid España","(40.028851, -3.606505)","Calle Bailén 19, Madrid España"
76.0,Taberna Más Al Sur,"Calle Santa Isabel Nº 35, 28012 Madrid España",Mediterránea,#34 de 10.414 restaurantes en Madrid,€€ - €€€,Calle Santa Isabel Nº 35,Madrid España,Calle Santa Isabel 35,"Calle Santa Isabel 35, Madrid España","(40.0316837, -3.5908706)","Calle Santa Isabel 35, Madrid España"
95.0,Casa Labra,"Calle Tetuan 12, 28013 Madrid España",Bar,#650 de 10.414 restaurantes en Madrid,€,Calle Tetuan 12,Madrid España,Calle Tetuan 12,"Calle Tetuan 12, Madrid España","(40.0395885, -3.6142713)","Calle Tetuan 12, Madrid España"
134.0,Parrilla El Gaucho,"Calle Tetuan 34, 28013 Madrid España",Restaurante de carne,#311 de 10.414 restaurantes en Madrid,€€ - €€€,Calle Tetuan 34,Madrid España,Calle Tetuan 34,"Calle Tetuan 34, Madrid España","(40.0395885, -3.6142713)","Calle Tetuan 34, Madrid España"
279.0,Santa Eulalia Boulangerie,"Calle Espejo 12, 28013 Madrid España",Confiterías/panaderías,N.º 23 de 477 cafeterías y teterías en Madrid,€€ - €€€,Calle Espejo 12,Madrid España,Calle Espejo 12,"Calle Espejo 12, Madrid España","(40.1384846, -3.4252641)","Calle Espejo 12, Madrid España"


In [9]:
import ast

In [10]:
def ast_literal (x):
    try:
        return ast.literal_eval(x)
    except:
        return x

In [11]:
df.coordinates = df.coordinates.apply(ast_literal)

In [12]:
def turn_around(x):
    b = (x[1], x[0])
    return list(b)

In [13]:
df["coordinates_v2"] = df.coordinates.apply(turn_around)

In [14]:
df.head()

,name,address,type,ranking,price,direccion,comunidad,street,street_2,coordinates,street_3,coordinates_v2
indice,,,,,,,,,,,,
413.0,El anciano rey de los vinos,"Calle Bailén 19, 28013 Madrid España",Mediterránea,#549 de 10.409 restaurantes en Madrid,€€ - €€€,Calle Bailén 19,Madrid España,Calle Bailén 19,"Calle Bailén 19, Madrid España","(40.028851, -3.606505)","Calle Bailén 19, Madrid España","[-3.606505, 40.028851]"
76.0,Taberna Más Al Sur,"Calle Santa Isabel Nº 35, 28012 Madrid España",Mediterránea,#34 de 10.414 restaurantes en Madrid,€€ - €€€,Calle Santa Isabel Nº 35,Madrid España,Calle Santa Isabel 35,"Calle Santa Isabel 35, Madrid España","(40.0316837, -3.5908706)","Calle Santa Isabel 35, Madrid España","[-3.5908706, 40.0316837]"
95.0,Casa Labra,"Calle Tetuan 12, 28013 Madrid España",Bar,#650 de 10.414 restaurantes en Madrid,€,Calle Tetuan 12,Madrid España,Calle Tetuan 12,"Calle Tetuan 12, Madrid España","(40.0395885, -3.6142713)","Calle Tetuan 12, Madrid España","[-3.6142713, 40.0395885]"
134.0,Parrilla El Gaucho,"Calle Tetuan 34, 28013 Madrid España",Restaurante de carne,#311 de 10.414 restaurantes en Madrid,€€ - €€€,Calle Tetuan 34,Madrid España,Calle Tetuan 34,"Calle Tetuan 34, Madrid España","(40.0395885, -3.6142713)","Calle Tetuan 34, Madrid España","[-3.6142713, 40.0395885]"
279.0,Santa Eulalia Boulangerie,"Calle Espejo 12, 28013 Madrid España",Confiterías/panaderías,N.º 23 de 477 cafeterías y teterías en Madrid,€€ - €€€,Calle Espejo 12,Madrid España,Calle Espejo 12,"Calle Espejo 12, Madrid España","(40.1384846, -3.4252641)","Calle Espejo 12, Madrid España","[-3.4252641, 40.1384846]"


In [15]:
type(df.coordinates[0])

tuple

In [16]:
type(df.coordinates_v2[0])

list

In [17]:
df.dtypes

name              object
address           object
type              object
ranking           object
price             object
direccion         object
comunidad         object
street            object
street_2          object
coordinates       object
street_3          object
coordinates_v2    object
dtype: object

In [44]:
def type_point(x):
    return {"type":"Point", "coordinates": [x[1], x[0]]}

In [45]:
df["type_point"] = df["coordinates"].apply(type_point)

In [46]:
df.head(2)

,name,address,type,ranking,price,direccion,comunidad,street,street_2,coordinates,street_3,coordinates_v2,type_point
indice,,,,,,,,,,,,,
413.0,El anciano rey de los vinos,"Calle Bailén 19, 28013 Madrid España",Mediterránea,#549 de 10.409 restaurantes en Madrid,€€ - €€€,Calle Bailén 19,Madrid España,Calle Bailén 19,"Calle Bailén 19, Madrid España","(40.028851, -3.606505)","Calle Bailén 19, Madrid España","[-3.606505, 40.028851]","{'type': 'Point', 'coordinates': [-3.606505, 4..."
76.0,Taberna Más Al Sur,"Calle Santa Isabel Nº 35, 28012 Madrid España",Mediterránea,#34 de 10.414 restaurantes en Madrid,€€ - €€€,Calle Santa Isabel Nº 35,Madrid España,Calle Santa Isabel 35,"Calle Santa Isabel 35, Madrid España","(40.0316837, -3.5908706)","Calle Santa Isabel 35, Madrid España","[-3.5908706, 40.0316837]","{'type': 'Point', 'coordinates': [-3.5908706, ..."


## Exportar a Mongo

In [47]:
client = MongoClient("localhost:27017")

In [48]:
db = client.get_database("PROJECT")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'PROJECT')

In [49]:
collection = db["Restaurants"]

In [50]:
collection.create_index([("type_point_", "2dsphere")])

'type_point__2dsphere'

In [51]:
data = df.to_dict(orient='records')

In [53]:
collection.insert_many(data)

In [54]:
ubicacion = input("¿Dónde estás?: ")
print(ubicacion)

¿Dónde estás?: Paseo de la chopera
Paseo de la chopera


In [30]:
tipo = input("¿Qué tipo de comida te apetece?: ")
print(tipo)

¿Qué tipo de comida te apetece?: fusion
fusion


In [ ]:
distance = int(input("¿Cuántos metros quieres desplazarte?: "))
print(distance)

In [ ]:
def get_coordenadas(address):
    """
    Esta función saca las coordenadas de la ciudad que le pases.
    Args: una ciudad (string).
    Return: Las coordeandas de la ciudad que le paso como argumento (latitud y longitud).
    """
    geolocator = Nominatim(user_agent="Ana")
    location = geolocator.geocode(query=address, exactly_one=True)
    localizacion = location[1]
    return [localizacion[1], localizacion[0]]

In [60]:
coordinate = get_coordenadas(ubicacion)

In [61]:
coordinate

[-3.7023735, 40.3962138]

In [82]:
def type_point(lista):
    return {"type":"Point", "coordinates": lista}

In [83]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'PROJECT')

In [84]:
restaurants = db.get_collection("Restaurants")

In [95]:
print(distance)

100000000000000


In [91]:
query_restaurants = {"coordinates_v2": {"$near": {"$geometry":{'type': 'Point', 'coordinates':coordinate}, "$maxDistance": distance}}}

In [92]:
query_restaurants

{'coordinates_v2': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': [-3.7023735, 40.3962138]},
   '$maxDistance': 100000000000000}}}

In [93]:
list(restaurants.find(query_restaurants))

[{'_id': ObjectId('61b89d2a5a0da3de9425bd20'),
  'name': 'Piantao',
  'address': 'Paseo la Chopera, 69, 28045 Madrid España',
  'type': 'Restaurante de carne',
  'ranking': '#832 de 10.409 restaurantes en Madrid',
  'price': '€€€€',
  'direccion': 'Paseo la Chopera 69',
  'comunidad': 'Madrid España',
  'street': 'Paseo la Chopera 69',
  'street_2': 'Paseo la Chopera 69, Madrid España',
  'coordinates': [40.394885, -3.7009705186583712],
  'street_3': 'Paseo la Chopera 69, Madrid España',
  'coordinates_v2': [-3.7009705186583712, 40.394885],
  'type_point': {'type': 'Point',
   'coordinates': [-3.7009705186583712, 40.394885]}},
 {'_id': ObjectId('61b89d2a5a0da3de9425bd24'),
  'name': 'Valdivieso Cocina Con Alma',
  'address': 'Calle Fray Luis de Leon, 7, 28012 Madrid España',
  'type': 'Mediterránea',
  'ranking': '#143 de 10.409 restaurantes en Madrid',
  'price': '€€ - €€€',
  'direccion': 'Calle Fray Luis de Leon 7',
  'comunidad': 'Madrid España',
  'street': 'Calle Fray Luis de Leo

In [94]:
len(list(restaurants.find(query_restaurants)))

500

1- Input preguntando donde estas
2- Input preguntando que radio quieren
3- Funcion que pasa de direccion a coordenadas (coordenadas al reves)
4- Funcion que me haga una query near a moongo y me devuelva todos los restaurantes en el radio  